<a href="https://colab.research.google.com/github/SHUDengwei/Pkmer-Docs/blob/main/03_%E5%A4%9A%E7%BB%B4%E5%BA%A6%E5%81%A5%E5%BA%B7%E8%AF%84%E5%88%86%E7%B3%BB%E7%BB%9F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 模块3｜多维度健康评分系统

**核心技能**：使用AI进行健康数据评分与分析

**目标**：定义评分规则，让Gemini根据规则对健康数据进行多维度评估

---

## 📚 本模块学习目标

在这个模块中，我们将：
1. ✅ 定义清晰的健康评分规则
2. ✅ 使用Gemini AI进行智能评分
3. ✅ 生成多维度健康评分报告
4. ✅ 获取AI提供的个性化改善建议
5. ✅ 输出格式化的控制台文本报告

---

## 🎯 评分体系设计

### 评分维度（每项0-100分）
- 🍎 **营养评分**：基于饮食均衡度、热量摄入、营养素比例
- 😴 **睡眠评分**：基于睡眠时长、睡眠质量、作息规律
- 🏃 **运动评分**：基于运动时长、运动频率、运动强度
- 😌 **压力评分**：基于情绪记录、压力水平

### 评级标准
- 🟢 **优秀**：85-100分
- 🔵 **良好**：70-84分
- 🟡 **一般**：60-69分
- 🟠 **需改善**：40-59分
- 🔴 **较差**：0-39分

### 权重分配
```
总分 = 营养评分 × 0.30 + 睡眠评分 × 0.30 + 运动评分 × 0.25 + 压力评分 × 0.15
```

**核心理念**：我们定义规则，让AI根据规则进行评分和分析！


## 1️⃣ 环境配置


In [ ]:
# 安装依赖
%pip install -q google-genai

import os
import json
from google import genai
from google.genai import types

# 配置API
try:
    from google.colab import userdata
    api_key = userdata.get('GEMINI_API_KEY')
except ImportError:
    api_key = os.getenv('GEMINI_API_KEY')

if not api_key:
    raise ValueError("❌ 请先配置 GEMINI_API_KEY")

client = genai.Client(api_key=api_key)
print("✅ Gemini API 配置成功！")


✅ Gemini API 配置成功！


## 2️⃣ 定义评分规则

我们将评分规则用自然语言清晰地表达，让AI理解并执行。


In [ ]:
# 定义评分规则
SCORING_RULES = """
你是一名专业的健康评估师，请根据以下规则对用户的健康数据进行多维度评分：

## 评分维度和标准

### 1. 😴 睡眠评分（0-100分）
**评分要素：**
- 睡眠时长（40分）：7-9小时最佳（40分），6-7或9-10小时（30分），5-6或10-11小时（20分），其他（10分）
- 睡眠质量（40分）：很好（40分），良好（32分），一般（24分），较差（16分），很差（8分）
- 夜间中断次数（20分）：0次（20分），1次（15分），2次（10分），3次以上（5分）

### 2. 🍎 营养评分（0-100分）
**评分要素：**
- 用餐规律性（30分）：三餐齐全（30分），两餐（20分），一餐或不规律（10分）
- 食物多样性（30分）：8种以上不同食物（30分），6-7种（24分），4-5种（18分），3种以下（12分）
- 健康度（40分）：
  * 健康食物：鸡胸肉、西兰花、糙米、沙拉、鱼、蔬菜、水果、全麦、牛奶、鸡蛋等
  * 不健康食物：油腻、油炸、奶茶、火锅、麻辣烫、过多外卖、烧烤等
  * 根据健康食物与不健康食物的比例评分

### 3. 🏃 运动评分（0-100分）
**评分要素：**
- 运动时长（50分）：60分钟以上（50分），30-60分钟（40分），15-30分钟（25分），1-15分钟（15分），无运动（0分）
- 运动强度（30分）：高强度（30分），中强度（24分），低强度（18分）
- 日常活动量/步数（20分）：10000步以上（20分），8000-10000步（18分），6000-8000步（16分），4000-6000步（12分），4000步以下（8分）

### 4. 😌 压力/情绪评分（0-100分）
**评分要素：**
- 心情评分（60分）：直接按用户的mood_score × 0.6计算
- 压力水平（40分）：低（40分），中（28分），高（16分），极高（8分）

## 评级标准
- 🟢 优秀：85-100分
- 🔵 良好：70-84分
- 🟡 一般：60-69分
- 🟠 需改善：40-59分
- 🔴 较差：0-39分

## 总分计算
总分 = 营养评分 × 0.30 + 睡眠评分 × 0.30 + 运动评分 × 0.25 + 压力评分 × 0.15

## 输出格式要求
请按照以下格式输出健康评分报告（纯文本）：

======================================================================
🏥 健康评分报告 - YYYY-MM-DD
======================================================================

📊 综合健康总分：XX/100（评级）

├─ 🍎 营养：XX分（评级）
│   详细分析和建议（2-3句话）

├─ 😴 睡眠：XX分（评级）
│   详细分析和建议（2-3句话）

├─ 🏃 运动：XX分（评级）
│   详细分析和建议（2-3句话）

└─ 😌 压力：XX分（评级）
    详细分析和建议（2-3句话）

----------------------------------------------------------------------
💡 优先改善建议：
1. 最需要改善的第1条建议
2. 最需要改善的第2条建议
3. 最需要改善的第3条建议

✨ 做得好的方面：
  • 第1个优点
  • 第2个优点
======================================================================
"""

print("✅ 评分规则已定义！")


✅ 评分规则已定义！


## 3️⃣ 准备健康数据

使用模块1解析的健康数据作为输入。


In [ ]:
# 小明周一的健康数据（来自模块1的输出）
monday_health_data = {
  "date": "2025-11-11",
  "sleep": {
    "bedtime": "23:30",
    "wake_time": "07:00",
    "duration_hours": 7.5,
    "quality": "一般",
    "interruptions": 0,
    "notes": "早上醒来感觉有点困"
  },
  "meals": [
    {
      "meal_type": "breakfast",
      "time": "08:00",
      "foods": ["煎饼果子", "豆浆"],
      "notes": "挺好吃的"
    },
    {
      "meal_type": "lunch",
      "time": "12:30",
      "foods": ["盖浇饭", "奶茶"],
      "notes": "有点油腻"
    },
    {
      "meal_type": "dinner",
      "time": "21:00",
      "foods": ["麻辣烫"],
      "notes": "外卖"
    }
  ],
  "exercise": {
    "activities": [],
    "total_duration_minutes": 0,
    "steps": 2500,
    "notes": "下午一直坐着开会"
  },
  "mood": {
    "stress_level": "高",
    "mood_score": 50,
    "notes": "工作压力有点大"
  }
}

# 小明周二的健康数据
tuesday_health_data = {
  "date": "2025-11-12",
  "sleep": {
    "bedtime": "00:00",
    "wake_time": "06:30",
    "duration_hours": 6.5,
    "quality": "一般",
    "interruptions": 0,
    "notes": "还挺累的"
  },
  "meals": [
    {
      "meal_type": "lunch",
      "time": "12:00",
      "foods": ["轻食沙拉", "鸡胸肉"],
      "notes": "感觉还挺健康的"
    },
    {
      "meal_type": "dinner",
      "time": "19:00",
      "foods": ["糙米饭", "清蒸鱼"],
      "notes": None
    }
  ],
  "exercise": {
    "activities": [
      {
        "type": "散步",
        "duration_minutes": 30,
        "intensity": "低"
      }
    ],
    "total_duration_minutes": 30,
    "steps": 7500,
    "notes": None
  },
  "mood": {
    "stress_level": "低",
    "mood_score": 80,
    "notes": "心情不错，工作也比较顺利"
  }
}

print("✅ 健康数据已加载！")
print(f"📅 周一数据：{monday_health_data['date']}")
print(f"📅 周二数据：{tuesday_health_data['date']}")


✅ 健康数据已加载！
📅 周一数据：2025-11-11
📅 周二数据：2025-11-12


## 4️⃣ 使用AI进行健康评分

现在让Gemini根据规则对健康数据进行评分。

In [ ]:
def score_health_data(health_data: dict) -> str:
    """
    使用Gemini AI对健康数据进行评分

    参数：
        health_data: 健康数据字典

    返回：
        格式化的评分报告文本
    """
    # 构建prompt
    prompt = f"""
{SCORING_RULES}

## 待评分的健康数据

```json
{json.dumps(health_data, ensure_ascii=False, indent=2)}
```

请根据上述规则对这份健康数据进行评分，并按照指定格式输出文本报告。
"""

    print(f"🤖 正在使用AI评估 {health_data['date']} 的健康数据...")

    # 调用API（不使用JSON模式，直接输出文本）
    response = client.models.generate_content(
        model='gemini-2.0-flash-exp',
        contents=prompt
    )

    print(f"✅ 评分完成！\n")

    return response.text

print("✅ 评分函数已定义！")


✅ 评分函数已定义！


In [ ]:
# 评估周一的健康数据并直接打印报告
monday_report = score_health_data(monday_health_data)
print(monday_report)

🤖 正在使用AI评估 2025-11-11 的健康数据...
✅ 评分完成！

```text
🏥 健康评分报告 - 2025-11-11

📊 综合健康总分：55.4/100（需改善）

├─ 🍎 营养：44分（🟠 需改善）
│   三餐规律，但食物选择不够健康，且多样性不足。建议增加蔬菜水果的摄入，减少油腻和外卖食品，并尝试更多种类的食物。

├─ 😴 睡眠：74分（🔵 良好）
│   睡眠时长尚可，但睡眠质量一般，夜间无中断。建议改善睡眠质量，可以尝试睡前放松活动，营造良好的睡眠环境。

├─ 🏃 运动：10分（🔴 较差）
│   运动量严重不足，步数过低，几乎没有运动。建议增加日常活动量，尝试进行一些简单的运动，如散步、跑步等。

└─ 😌 压力：38分（🔴 较差）
    压力水平较高，心情也较为低落。建议寻找缓解压力的方法，例如冥想、听音乐、与朋友交流等，并注意调节情绪。

----------------------------------------------------------------------
💡 优先改善建议：
1. 增加运动量，每天至少进行30分钟的运动。
2. 改善饮食结构，多摄入蔬菜水果，减少不健康食品。
3. 寻找有效缓解压力的方法，保持积极乐观的心态。

✨ 做得好的方面：
  • 睡眠时长基本达标
  • 夜间睡眠未被打断
```


## 7️⃣ 示例：评估小明周二的健康状况

In [ ]:
# 评估周二的健康数据并直接打印报告
tuesday_report = score_health_data(tuesday_health_data)
print(tuesday_report)

🤖 正在使用AI评估 2025-11-12 的健康数据...
✅ 评分完成！

```text
🏥 健康评分报告 - 2025-11-12

📊 综合健康总分：74/100（良好）

├─ 🍎 营养：78分（良好）
│   您的一日两餐，健康食物选择较好，但用餐规律性有待加强，建议增加早餐，并丰富食物种类。

├─ 😴 睡眠：64分（一般）
│   您的睡眠时长稍短，睡眠质量一般，但夜间中断次数为0，建议尽量保证7-9小时的睡眠时长，提高睡眠质量。

├─ 🏃 运动：68分（一般）
│   您的运动时长和步数尚可，但运动强度较低，建议增加运动强度，并尽可能达到60分钟以上的运动时长。

└─ 😌 压力：62分（一般）
    您的心情不错，压力水平较低，请继续保持，并注意劳逸结合，避免压力过大。

----------------------------------------------------------------------
💡 优先改善建议：
1. 增加睡眠时长，保证7-9小时的睡眠。
2. 增加运动强度，尝试进行中高强度运动。
3. 增加一餐，保证三餐齐全，丰富食物种类。

✨ 做得好的方面：
  • 心情不错，压力水平较低
  • 夜间睡眠中断次数少
```

**评分细则解释：**

*   **睡眠评分：**
    *   睡眠时长：6.5小时，对应30分
    *   睡眠质量：一般，对应24分
    *   夜间中断次数：0次，对应20分
    *   总分：30 + 24 + 20 = 74分
*   **营养评分：**
    *   用餐规律性：两餐，对应20分
    *   食物多样性：午餐2种，晚餐2种，共4种，对应18分
    *   健康度：午餐轻食沙拉、鸡胸肉，晚餐糙米饭、清蒸鱼，基本健康，评40分
    *   总分：20 + 18 + 40 = 78分
*   **运动评分：**
    *   运动时长：30分钟，对应40分
    *   运动强度：低强度，对应18分
    *   日常活动量/步数：7500步，对应16分
    *   总分：40 + 18 + 16 = 74分
*   **压力/情绪评分：**
    *   心情评分：80 * 0.6 = 48分
    *   压力水平：低，

## 8️⃣ （可选）让AI对比分析两天的数据

如果需要对比分析，可以将两天的报告一起发给AI进行对比。

In [ ]:
# 方法：让AI对比两天的报告
comparison_prompt = f"""
请对比以下两天的健康评分报告，分析健康状况的变化趋势：

## 周一报告
{monday_report}

## 周二报告
{tuesday_report}

请分析：
1. 哪些维度有改善，哪些维度有下降
2. 总体健康趋势如何
3. 给出针对性的建议

请用简洁的文本格式输出对比分析。
"""

print("🤖 正在生成对比分析...")
comparison_response = client.models.generate_content(
    model='gemini-2.0-flash-exp',
    contents=comparison_prompt
)

print("\n" + "=" * 70)
print("📈 两日健康对比分析")
print("=" * 70)
print(comparison_response.text)
print("=" * 70)


🤖 正在生成对比分析...

📈 两日健康对比分析
**健康状况对比分析 (周一 vs 周二):**

1.  **改善的维度：**
    *   **总体健康总分:** 从55.4 (需改善) 提升到 74 (良好)。
    *   **营养:** 从 44 (需改善) 提升到 78 (良好)。
    *   **运动:** 从 10 (较差) 提升到 68 (一般)。
    *   **压力:** 从 38 (较差) 提升到 62 (一般)。

2.  **下降的维度：**
    *   **睡眠:** 从 74 (良好) 下降到 64 (一般)，主要由于睡眠时长缩短和睡眠质量下降。

3.  **总体健康趋势：**
    *   总体呈现显著改善的趋势，各个维度都取得进步，综合评分大幅提升。

4.  **针对性建议：**
    *   **保持并加强改善：** 继续保持周二在饮食、运动和压力管理方面的积极做法。
    *   **关注睡眠质量与时长：** 虽然夜间中断次数少，但睡眠时长需增加到7-9小时，并尝试改善睡眠质量。回顾周一睡眠好，找出影响睡眠质量的因素。
    *   **三餐规律：** 增加早餐，形成更规律的饮食习惯。
    *   **运动强度提升：** 在现有运动基础上，逐步增加运动强度。
    *   **劳逸结合：** 保持良好情绪的同时，注意劳逸结合，避免压力过大。

